<a href="https://colab.research.google.com/github/saykim/006958/blob/master/kera_kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [207]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.datasets import load_boston
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler


In [208]:
data = load_boston()

In [209]:
x_data = pd.DataFrame(data = data['data'], columns=data['feature_names'])
y_data = pd.DataFrame(data['target'])

In [231]:
x_data.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10


In [211]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=.3, random_state=44, shuffle=True)

In [212]:
scaler = MinMaxScaler()

In [220]:
x_train_sc = scaler.fit_transform(x_train)
x_test_sc = scaler.transform(x_test)

x_train = pd.DataFrame(data=x_train_sc, columns=x_train.columns, index=x_train.index)
x_test = pd.DataFrame(data=x_test_sc, columns=x_test.columns, index = x_test.index)



In [221]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping

In [222]:
es = EarlyStopping(monitor='val_loss', patience=5)

In [223]:
def get_model():
    input = Input(shape=(13,))

    x = Dense(32, activation='relu')(input)
    x = Dense(64, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(1)(x)

    model = Model(input, output)
    model.compile(optimizer='adam', loss = 'mse', metrics=['mae'])
    return model



In [224]:
kfold = KFold(n_splits=3, random_state=44, shuffle=True)

In [225]:
mae_list = []

for train_index, val_index in kfold.split(x_train):
    # 해당 인덱스는 무작위로 생성됩니다.
    # 무작위로 생성해주는 것은 과대적합을 피할 수 있는 좋은 방법입니다.
    x_train_fold, x_val_fold = x_train.iloc[train_index], x_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    model = get_model()
    model.fit(x_train_fold, y_train_fold, epochs=100, batch_size=10, verbose=1,callbacks=es, validation_data=(x_val_fold, y_val_fold))
    
    _, test_mae = model.evaluate(x_test, y_test)
    mae_list.append(test_mae)

print(mae_list)


Epoch 1/100
24/24 [==============================] - 1s 13ms/step - loss: 543.9337 - mae: 21.5767 - val_loss: 393.0091 - val_mae: 16.8291
Epoch 2/100
24/24 [==============================] - 0s 4ms/step - loss: 202.8872 - mae: 11.1232 - val_loss: 155.3945 - val_mae: 9.0642
Epoch 3/100
24/24 [==============================] - 0s 5ms/step - loss: 99.2823 - mae: 6.9673 - val_loss: 99.4727 - val_mae: 7.6151
Epoch 4/100
24/24 [==============================] - 0s 4ms/step - loss: 58.6366 - mae: 5.8408 - val_loss: 83.5927 - val_mae: 6.6295
Epoch 5/100
24/24 [==============================] - 0s 4ms/step - loss: 39.7511 - mae: 4.7734 - val_loss: 79.1817 - val_mae: 5.9667
Epoch 6/100
24/24 [==============================] - 0s 4ms/step - loss: 48.0026 - mae: 4.9816 - val_loss: 68.5026 - val_mae: 5.5382
Epoch 7/100
24/24 [==============================] - 0s 5ms/step - loss: 45.5435 - mae: 4.7483 - val_loss: 62.7541 - val_mae: 5.0521
Epoch 8/100
24/24 [==============================] - 0s 4ms/s

In [230]:
mae_list

[3.1656134128570557, 2.9836905002593994, 2.759782075881958]

In [104]:
model.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 13)]              0         
_________________________________________________________________
dense_62 (Dense)             (None, 32)                448       
_________________________________________________________________
dense_63 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_64 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_65 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 33        
Total params: 8,833
Trainable params: 8,833
Non-trainable params: 0
________________________________________________________

In [105]:
model.fit(x_data, y_data, epochs=100, batch_size=10, verbose=1,callbacks=es, validation_data=(x_test, y_test))

Epoch 1/100
51/51 [==============================] - 1s 6ms/step - loss: 1332.6638 - mae: 26.4836 - val_loss: 81.4126 - val_mae: 6.2119
Epoch 2/100
51/51 [==============================] - 0s 3ms/step - loss: 59.5273 - mae: 5.6423 - val_loss: 71.5017 - val_mae: 5.6992
Epoch 3/100
51/51 [==============================] - 0s 2ms/step - loss: 58.1320 - mae: 5.5339 - val_loss: 57.2966 - val_mae: 5.1311
Epoch 4/100
51/51 [==============================] - 0s 3ms/step - loss: 74.6804 - mae: 5.9751 - val_loss: 61.0766 - val_mae: 5.1722
Epoch 5/100
51/51 [==============================] - 0s 3ms/step - loss: 50.4500 - mae: 5.1667 - val_loss: 54.9040 - val_mae: 4.8822
Epoch 6/100
51/51 [==============================] - 0s 3ms/step - loss: 55.7966 - mae: 5.2699 - val_loss: 52.7269 - val_mae: 5.3028
Epoch 7/100
51/51 [==============================] - 0s 2ms/step - loss: 55.6801 - mae: 5.2274 - val_loss: 55.5086 - val_mae: 5.0127
Epoch 8/100
51/51 [==============================] - 0s 3ms/step -

In [107]:
model.history.history['loss']

[495.4952087402344,
 63.674564361572266,
 63.73788070678711,
 59.41636657714844,
 57.4622802734375,
 54.2923469543457,
 59.45656204223633,
 55.34852600097656,
 51.87218475341797,
 54.528564453125,
 54.027381896972656,
 53.331207275390625,
 52.22025680541992,
 47.78224182128906,
 43.866878509521484,
 52.907371520996094,
 48.88383102416992,
 56.23727798461914,
 46.77631378173828,
 43.437564849853516,
 40.115020751953125,
 46.845420837402344,
 45.88404083251953,
 37.85439682006836,
 36.760345458984375,
 34.06074142456055,
 31.71523666381836,
 32.46758270263672,
 31.93307876586914,
 30.62209129333496,
 30.576892852783203,
 33.03178024291992,
 30.891206741333008,
 28.789621353149414,
 27.451139450073242,
 27.907499313354492,
 27.53173828125,
 29.81560707092285,
 30.639543533325195,
 24.505876541137695,
 28.0772705078125,
 24.619733810424805,
 26.69271469116211,
 23.359025955200195,
 23.556974411010742,
 21.979705810546875]